In [2]:
!pip install gym

  Created wheel for gym: filename=gym-0.18.0-py3-none-any.whl size=1656454 sha256=2394e36a4f9c9122f125ee54a054d694d62f0f69bbd43287a784ea43b15047f9
  Stored in directory: c:\users\dimna\appdata\local\pip\cache\wheels\99\f7\e3\d6f0f120ac047c1e5de2ae34930e7bf6e8de1c7a4d5fa68555
Successfully built gym


In [4]:
import gym
env = gym.make("MountainCar-v0")
observation = env.reset()
for _ in range(1000):
  env.render()
  action = env.action_space.sample() # your agent here (this takes random actions)
  observation, reward, done, info = env.step(action)

  if done:
    observation = env.reset()
env.close()